# Úkol klasifikace textu

V tomto modulu začneme jednoduchým úkolem klasifikace textu na základě datasetu **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**: budeme klasifikovat nadpisy zpráv do jedné ze 4 kategorií: Svět, Sport, Byznys a Věda/Technika.

## Dataset

Pro načtení datasetu použijeme API **[TensorFlow Datasets](https://www.tensorflow.org/datasets)**.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Nyní můžeme přistupovat k trénovací a testovací části datové sady pomocí `dataset['train']` a `dataset['test']`:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Pojďme vytisknout prvních 10 nových titulků z našeho datového souboru:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Vektorizace textu

Nyní potřebujeme převést text na **čísla**, která mohou být reprezentována jako tensory. Pokud chceme reprezentaci na úrovni slov, musíme udělat dvě věci:

* Použít **tokenizér** k rozdělení textu na **tokeny**.
* Vytvořit **slovník** těchto tokenů.

### Omezení velikosti slovníku

V příkladu s datasetem AG News je velikost slovníku poměrně velká, více než 100 tisíc slov. Obecně řečeno, nepotřebujeme slova, která se v textu vyskytují jen zřídka — pouze několik vět je bude obsahovat a model se z nich nic nenaučí. Proto má smysl omezit velikost slovníku na menší počet tím, že předáme argument konstruktoru vektorizéru:

Oba tyto kroky lze zvládnout pomocí vrstvy **TextVectorization**. Pojďme vytvořit objekt vektorizéru a poté zavolat metodu `adapt`, která projde veškerý text a vytvoří slovník:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Poznámka**: Používáme pouze podmnožinu celého datasetu k vytvoření slovníku. Děláme to proto, abychom zrychlili dobu zpracování a nemuseli jste dlouho čekat. Tím však podstupujeme riziko, že některá slova z celého datasetu nebudou zahrnuta do slovníku a budou během trénování ignorována. Použití celé velikosti slovníku a zpracování celého datasetu během `adapt` by mělo zvýšit konečnou přesnost, ale ne výrazně.

Nyní můžeme přistoupit k samotnému slovníku:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Pomocí vektorizéru můžeme snadno zakódovat jakýkoli text do sady čísel:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Reprezentace textu pomocí Bag-of-words

Protože slova nesou význam, někdy můžeme pochopit smysl textu jen tím, že se podíváme na jednotlivá slova, bez ohledu na jejich pořadí ve větě. Například při klasifikaci zpráv slova jako *počasí* a *sníh* pravděpodobně naznačují *předpověď počasí*, zatímco slova jako *akcie* a *dolar* by spíše ukazovala na *finanční zprávy*.

Reprezentace vektoru **Bag-of-words** (BoW) je nejjednodušší tradiční vektorová reprezentace na pochopení. Každé slovo je spojeno s indexem vektoru a prvek vektoru obsahuje počet výskytů každého slova v daném dokumentu.

![Obrázek ukazující, jak je reprezentace Bag-of-words uložena v paměti.](../../../../../translated_images/cs/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Na BoW můžete také nahlížet jako na součet všech one-hot-encoded vektorů pro jednotlivá slova v textu.

Níže je uveden příklad, jak vytvořit reprezentaci Bag-of-words pomocí python knihovny Scikit Learn:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Můžeme také použít Keras vektorizér, který jsme definovali výše, převést každé číslo slova na one-hot kódování a všechny tyto vektory sečíst:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Poznámka**: Může vás překvapit, že výsledek se liší od předchozího příkladu. Důvodem je, že v příkladu s Kerasem délka vektoru odpovídá velikosti slovníku, který byl vytvořen z celého datasetu AG News, zatímco v příkladu se Scikit Learn jsme slovník vytvořili z ukázkového textu za běhu.


## Trénování klasifikátoru BoW

Nyní, když jsme se naučili, jak vytvořit reprezentaci textu pomocí "bag-of-words" (BoW), pojďme natrénovat klasifikátor, který ji využívá. Nejprve musíme převést náš dataset na reprezentaci "bag-of-words". Toho lze dosáhnout použitím funkce `map` následujícím způsobem:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Nyní definujme jednoduchou klasifikační neuronovou síť, která obsahuje jednu lineární vrstvu. Velikost vstupu je `vocab_size` a velikost výstupu odpovídá počtu tříd (4). Protože řešíme klasifikační úlohu, konečná aktivační funkce je **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Vzhledem k tomu, že máme 4 třídy, přesnost nad 80 % je dobrý výsledek.

## Trénování klasifikátoru jako jedné sítě

Protože je vektorizér také vrstvou Keras, můžeme definovat síť, která ho zahrnuje, a trénovat ji od začátku do konce. Tímto způsobem není potřeba vektorizovat dataset pomocí `map`, stačí předat původní dataset jako vstup do sítě.

> **Note**: Stále bychom museli aplikovat mapy na náš dataset, abychom převedli pole ze slovníků (jako `title`, `description` a `label`) na dvojice. Nicméně při načítání dat z disku můžeme dataset rovnou vytvořit ve požadované struktuře.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Bigramy, trigramy a n-gramy

Jedním z omezení přístupu bag-of-words je, že některá slova jsou součástí víceslovných výrazů. Například slovo „hot dog“ má úplně jiný význam než slova „hot“ a „dog“ v jiných kontextech. Pokud bychom slova „hot“ a „dog“ vždy reprezentovali stejnými vektory, mohlo by to náš model zmást.

Abychom tento problém vyřešili, často se v metodách klasifikace dokumentů používají **reprezentace n-gramů**, kde je frekvence každého slova, dvojice slov nebo trojice slov užitečnou vlastností pro trénování klasifikátorů. Například v bigramových reprezentacích přidáme do slovníku všechny dvojice slov, kromě původních slov.

Níže je uveden příklad, jak vytvořit bigramovou reprezentaci bag-of-words pomocí Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Hlavní nevýhodou přístupu n-gramů je, že velikost slovníku začíná růst extrémně rychle. V praxi je potřeba kombinovat reprezentaci n-gramů s technikou redukce dimenzionality, jako jsou *embeddingy*, o kterých budeme diskutovat v další jednotce.

Abychom použili reprezentaci n-gramů v našem datasetu **AG News**, musíme předat parametr `ngrams` do našeho konstruktoru `TextVectorization`. Délka slovníku bigramů je **výrazně větší**, v našem případě je to více než 1,3 milionu tokenů! Proto má smysl omezit počet bigramových tokenů na nějaké rozumné číslo.

Mohli bychom použít stejný kód jako výše k trénování klasifikátoru, ale bylo by to velmi neefektivní z hlediska paměti. V další jednotce budeme trénovat klasifikátor bigramů pomocí embeddingů. Mezitím můžete experimentovat s trénováním klasifikátoru bigramů v tomto notebooku a zjistit, zda dokážete dosáhnout vyšší přesnosti.


## Automatický výpočet BoW vektorů

V předchozím příkladu jsme BoW vektory počítali ručně sčítáním jednohotových kódování jednotlivých slov. Nicméně nejnovější verze TensorFlow nám umožňuje vypočítat BoW vektory automaticky pomocí předání parametru `output_mode='count` konstruktoru vektorizéru. To výrazně usnadňuje definování a trénování našeho modelu:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Frekvence termínů - inverzní frekvence dokumentů (TF-IDF)

V reprezentaci BoW jsou výskyty slov váženy stejnou technikou bez ohledu na samotné slovo. Je však zřejmé, že častá slova jako *a* a *v* jsou pro klasifikaci mnohem méně důležitá než specializované termíny. Ve většině úloh NLP jsou některá slova relevantnější než jiná.

**TF-IDF** znamená **frekvence termínů - inverzní frekvence dokumentů**. Jedná se o variaci na bag-of-words, kde místo binární hodnoty 0/1, která označuje výskyt slova v dokumentu, je použita hodnota s plovoucí desetinnou čárkou, která souvisí s frekvencí výskytu slova v korpusu.

Formálněji je váha $w_{ij}$ slova $i$ v dokumentu $j$ definována jako:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
kde
* $tf_{ij}$ je počet výskytů $i$ v $j$, tj. hodnota BoW, kterou jsme viděli dříve
* $N$ je počet dokumentů v kolekci
* $df_i$ je počet dokumentů obsahujících slovo $i$ v celé kolekci

Hodnota TF-IDF $w_{ij}$ roste úměrně s počtem výskytů slova v dokumentu a je korigována počtem dokumentů v korpusu, které slovo obsahují, což pomáhá zohlednit skutečnost, že některá slova se objevují častěji než jiná. Například pokud se slovo objeví *v každém* dokumentu v kolekci, pak $df_i=N$ a $w_{ij}=0$, a tyto termíny by byly zcela ignorovány.

TF-IDF vektorizaci textu můžete snadno vytvořit pomocí Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

V Keras může vrstva `TextVectorization` automaticky vypočítat TF-IDF frekvence pomocí předání parametru `output_mode='tf-idf'`. Pojďme zopakovat kód, který jsme použili výše, abychom zjistili, zda použití TF-IDF zvyšuje přesnost:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Závěr

I když reprezentace TF-IDF poskytují váhové frekvence různým slovům, nejsou schopny reprezentovat význam nebo pořadí. Jak slavný lingvista J. R. Firth řekl v roce 1935: „Úplný význam slova je vždy kontextový a žádná studie významu bez kontextu nemůže být brána vážně.“ Později v kurzu se naučíme, jak zachytit kontextové informace z textu pomocí jazykového modelování.



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby pro automatický překlad [Co-op Translator](https://github.com/Azure/co-op-translator). I když se snažíme o přesnost, mějte prosím na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace se doporučuje profesionální lidský překlad. Neodpovídáme za žádné nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
